#                                        Manual Match Automation Script

## Importing all the required packages

In [46]:
import os
import pandas as pd
import re
import numpy as np
import glob
from nltk.corpus import stopwords
from xlrd import XLRDError
# nltk_stop_words = stopwords.words('english')
from stop_words import get_stop_words
from num2words import num2words 

from difflib import get_close_matches
from re import sub
# path='/Users/simranpal20gmail.com/Documents/Apple/Data/Week13/Manual_Match'

## Reading and making Transaction files ready

In [47]:
Week_number="Week26"
admins_divide=3
admins_divide_dict={1:'Ciara',2:'Neil',3:'Simran'}
# admins_divide_dict={1:'Ciara',2:'Neil'}

## 73 partners Analysis

In [48]:
path="/Users/simranpal20gmail.com/Documents/Apple/Data/"+Week_number+"/Manual_Match/73_Partners"
os.chdir(path)
Partners_73_file=[f for f in os.listdir(path) if not f.startswith('.')]
Partners_73_df = pd.read_excel(Partners_73_file[0],'STW Baseline - 73 Partners')
Partners_73_ls=Partners_73_df['Apple HQ ID'].tolist()

## Post TARS File

In [49]:
# path='/Users/simranpal20gmail.com/Documents/Apple/Data/Week14/Manual_Match/Post_TARS'
path="/Users/simranpal20gmail.com/Documents/Apple/Data/"+Week_number+"/Manual_Match/Post_TARS"
os.chdir(path)
post_tars_file=[f for f in os.listdir(path) if not f.startswith('.')]
post_tars_df = pd.read_excel(post_tars_file[0],'Transactions')
post_tars_df=post_tars_df[post_tars_df['RESELLER HQ APPLE ID'].isin(Partners_73_ls)]
post_tars_df['END CUSTOMER NAME']=post_tars_df['END CUSTOMER NAME'].astype(str)

post_tars_df['name_ctry'] = post_tars_df[['END CUSTOMER NAME', 'END CUSTOMER COUNTRY']].apply(lambda x: ''.join(x), axis=1)
status_ls=['For Review','Pre Parked']
match_status_ls=['Exact Match']

# status_ls=['Approved']
# trans_pivot=pd.DataFrame({'ST_QTY':post_tars_df[post_tars_df['STATUS'].isin(status_ls)].groupby(['name_ctry'])['REPORTED QUANTITY'].sum()}).reset_index()
# trans_pivot=pd.DataFrame({'ST_QTY':post_tars_df[post_tars_df['STATUS'].isin(status_ls)].groupby(['name_ctry','STATUS'])['REPORTED QUANTITY'].count()}).reset_index()
# trans_pivot=post_tars_df[post_tars_df['STATUS'].isin(status_ls) & post_tars_df['END CUSTOMER NAME (MATCHED)'].isna()].groupby(['name_ctry','STATUS']).agg(Sum_Rpt_Qty=('REPORTED QUANTITY', 'sum'), Transaction_Counts=('name_ctry', 'count'))
trans_pivot=post_tars_df[post_tars_df['STATUS'].isin(status_ls) & ~post_tars_df['MATCH STATUS'].isin(match_status_ls)].groupby(['name_ctry','STATUS','MATCH STATUS']).agg(Sum_Rpt_Qty=('REPORTED QUANTITY', 'sum'), Transaction_Counts=('name_ctry', 'count'))
trans_pivot.reset_index(level=0, inplace=True)
trans_pivot.reset_index(level=0, inplace=True)
trans_pivot.reset_index(level=0, inplace=True)
trans_pivot['code']=trans_pivot['name_ctry'].str[-2:]
trans_pivot['Names']=trans_pivot['name_ctry'].str[:-2]

df1=trans_pivot.copy()
actual_df=df1.copy()

## Writing the Files for Auto_Match case

In [50]:
status_ls=['For Review','Approved']
match_status_ls=['Exact Match']

trans_pivot_1=post_tars_df[post_tars_df['STATUS'].isin(status_ls) & post_tars_df['MATCH STATUS'].isin(match_status_ls)].groupby(['name_ctry','END CUSTOMER NAME','END CUSTOMER NAME (MATCHED)','END CUSTOMER COUNTRY',
                                                                                                                               'END CUSTOMER COUNTRY (MATCHED)','STATUS']).agg(Sum_Rpt_Qty=('REPORTED QUANTITY', 'sum'), Transaction_Counts=('name_ctry', 'count'))
trans_pivot_1.reset_index(level=0, inplace=True)
trans_pivot_1.reset_index(level=0, inplace=True)
trans_pivot_1.reset_index(level=0, inplace=True)
trans_pivot_1.reset_index(level=0, inplace=True)
trans_pivot_1.reset_index(level=0, inplace=True)
trans_pivot_1.reset_index(level=0, inplace=True)

trans_pivot_1_cols_ls = ['END CUSTOMER NAME', 'END CUSTOMER NAME (MATCHED)', 'END CUSTOMER COUNTRY',
                'END CUSTOMER COUNTRY (MATCHED)','Transaction_Counts','Sum_Rpt_Qty','STATUS']
# trans_pivot_1 = trans_pivot_1.reindex(columns=trans_pivot_1_cols_ls)
trans_pivot_1 = trans_pivot_1[trans_pivot_1_cols_ls]

admins_divide_dict1={1:'Ciara_auto',2:'Neil_auto',3:'Simran_auto'}
# admins_divide_dict1={1:'Ciara_auto',2:'Simran_auto'}
path="/Users/simranpal20gmail.com/Documents/Apple/Data/"+Week_number+"/Manual_Match"
os.chdir(path)
Ciara_auto = pd.ExcelWriter('Auto_Match_Ciara.xlsx')
Neil_auto = pd.ExcelWriter('Auto_Match_Neil.xlsx')
Simran_auto = pd.ExcelWriter('Auto_Match_Simran.xlsx')

for status in status_ls:
    trans_pivot1_df=trans_pivot_1[trans_pivot_1['STATUS']==status]
    admins_divide1_np=np.array_split(trans_pivot1_df, admins_divide)
    for divide in range(0,admins_divide):
        dynamic_var=admins_divide_dict1[divide+1]
        admins_divide1_np[divide].to_excel(eval(dynamic_var), sheet_name=status,index=False)
Ciara_auto.save()
Neil_auto.save()
Simran_auto.save()

In [51]:
print("Total Unique End Customer Name Unmatched:",trans_pivot.shape[0])
print("Total Transaction with Unmatched records:",trans_pivot['Transaction_Counts'].sum())

Total Unique End Customer Name Unmatched: 6172
Total Transaction with Unmatched records: 12975


## PCH File

In [52]:
# path='/Users/simranpal20gmail.com/Documents/Apple/Data/Week14/Manual_Match/PCH'
path="/Users/simranpal20gmail.com/Documents/Apple/Data/"+Week_number+"/Manual_Match/PCH"
os.chdir(path)
try:
    pch_file=[f for f in os.listdir(path) if not f.startswith('.')]
    pch_df = pd.read_excel(pch_file[0])
except XLRDError:
    pch_df = pd.read_excel(pch_file[1])
pch_df.columns = pch_df.iloc[0]
pch_df = pch_df[1:]

pch_columns_ls=['Country','Child CRM NICE ID','Child Name','Parent Name']
pch_df=pch_df[pch_columns_ls]
df2=pd.DataFrame({'count' : pch_df.groupby(['Country','Child CRM NICE ID','Child Name','Parent Name']).size()}).reset_index()
# df2=pch_df.copy()

## Write Country Code File for BOX

In [53]:
# path='/Users/simranpal20gmail.com/Documents/Apple/Data/Week14/Manual_Match/Country_codes'
path="/Users/simranpal20gmail.com/Documents/Apple/Data/"+Week_number+"/Manual_Match/Country_codes"
os.chdir(path)
cc_write = pd.ExcelWriter('Simran_ManualMatch_ctry_codes.xlsx')

# simran_code_ls=['IT','DK','IN','FR','AT','FI','RU','ES','CZ','PL','CH','TR','PT','RO','LU','LT']
simran_code_ls=actual_df.code.value_counts().index.tolist()
simran_code_cols=['name_ctry','Sum of Qty','Matching Status','Child Name','Parent Name','Nice ID']
simran_code_df = pd.DataFrame(columns=simran_code_cols)


for cc_code in simran_code_ls:
    simran_code_df = pd.DataFrame(columns=simran_code_cols)
    temp_code_df=trans_pivot[trans_pivot.code==cc_code][['name_ctry','Sum_Rpt_Qty','STATUS','Transaction_Counts']]
    temp_code_df['Matching Status']="Not Matched"
    temp_code_df["Child Name"] = ""
    temp_code_df["Parent Name"] = ""
    temp_code_df["Nice ID"] = ""
#     simran_code_df=simran_code_df.append(temp_code_df)
#     simran_code_df.to_excel(cc_write, sheet_name=str(cc_code))
    temp_code_df.to_excel(cc_write, sheet_name=str(cc_code),index=False)
cc_write.save()

## File Cleansing

In [54]:
path='/Users/simranpal20gmail.com/Documents/Apple/Data'
os.chdir(path)

xls = pd.ExcelFile('EMEIA_Country_Code.xlsx')
emeia_codes = pd.read_excel(xls)

# strip white space from the column
df2['Country'] = df2['Country'].str.strip()
emeia_codes['CountryName']=emeia_codes['CountryName'].str.strip()
emeia_codes['ISO']=emeia_codes['ISO'].str.strip()
df2['Country_code']=df2.merge(emeia_codes, how='left',left_on='Country', right_on='CountryName')['ISO']

In [55]:
def Country_wise_stop_words(df1_sw):
    unique_country_code_ls=actual_df.code.value_counts().index.tolist()
    for code in unique_country_code_ls:
        if code in ['GB','IE','IN'] :
            code_en=code
            code='EN'
        try:
            stop_words_en = get_stop_words('en')
            stop_words = get_stop_words(code.lower())
            if code ==  'EN':
                code=code_en
            df1_sw.loc[df1_sw['code'] == code, 'Names'] = df1_sw[df1_sw.code==code]['Names'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop_words]))
            df1_sw.loc[df1_sw['code'] == code, 'Names'] = df1_sw[df1_sw.code==code]['Names'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop_words_en]))
        except:
            continue
    return df1_sw['Names']

In [56]:
# Lower case the end customer names in comparing file  
df1['Names']=df1['Names'].str.lower()

# Lower case the end customer names in PCH file
df2['Child Name']=df2['Child Name'].str.lower()

# removes numeric values from the df1 dataframe
# df1['Names'] = df1['Names'].str.replace('\d+', '')
# df1['Names'] = df1['Names'].str.replace('\b\d+\b *|\b[a-z]\b *', '')
# df2['Child Name'] = df2['Child Name'].str.replace('\d+', '')

# removes the character with less than 2 characters in df1
# df1['Names']=df1.Names.str.replace(r'\b(\w{1,1})\b', '')

# removes punctuations from df1
df1["Names"] = df1['Names'].str.replace('[^\w\s]','')

# strip white space from the column
df1['Names'] = df1['Names'].str.strip()

df1['Names'] = Country_wise_stop_words(df1)
df1_clean=df1.copy()

## Stop_words for SHEET_1

In [57]:
# stop_words_ls=[]
# stop_word_dic_1={}
# stop_word_dic_1["General"]=['solutions','solution','service','services','management','holding',
#                      'bank','technology','technologies','consulting','business','private','limited','software','hospital',
#                      'tech','industrial','corporation','company','europe','city','group','IT','region','international',
#                        'healthcare','community','east','west','north','marketing','global']
# stop_word_dic_1["AT"]=['austria','it','ag','kg','gmbh','ges','wien','österreich']
# stop_word_dic_1["BE"]=['belgium','bvba','stad','sa','de','la','nv','media']
# stop_word_dic_1["BG"]=[]
# stop_word_dic_1["CH"]=['sa','ag','schweiz','suisse','gmbh']
# stop_word_dic_1["CZ"]=['czech','banka','spol','česká','cz']
# stop_word_dic_1["DE"]=['germany','stadtwerke','stadt','krankenhaus','gmbh','ggmbh','sparkasse','deutschland','kliniken','berlin',
#                     'polizeipräsidium','gesellschaft','media','münchen','matt','klinikum','deutsche',
#                  'aktiengesellschaft','essen','pfalz','it','ag','kg','co.','co','für','st','des','der','und','nord',
#                 'eg','am','von','se','ohg','die','bundes','dresden','internet','and','springer','bau','bundesministerium',
#                     'nrw','db','gemeinnützige','kgaa','kreissparkasse','verlagsgesellschaft','mannheim','bosch','informatik',
#                     'mbh','gmb','sachsen','gbr','stuttgart','rundfunk','digital','bayerische','stiftung','deutsches',
#                     'ev','rhein','zweckverband','man','verlag','energie','national','axel','car','impact']
# stop_word_dic_1["DK"]=['denmark','danish','dansk','og','aps','danmark','kommune','as']
# stop_word_dic_1["EG"]=[]
# stop_word_dic_1["ES"]=[]
# stop_word_dic_1["FI"]=['finland','turun','oy','seudun','ab','uudenmaan','oyj','suomen']
# stop_word_dic_1["FO"]=[]
# stop_word_dic_1["FR"]=['france','epargne','systemes','editions','paris','sa','et','gie','groupe','de','la','da','sas','caisse',
#                     'pour','it','snc','commune','francaise','des','se','ministere','sud','distribution','loisirs',
#                     'centre','le','sarl']
# stop_word_dic_1["GB"]=['uk','united', 'kingdom','u.k.','london','it','plc','the','national','ltd','northern','authority','systems',
#                     'emea','network','partners','fire','care','england','and','council','of','new','gb','nhs','trust','media',
#                     'rescue','health','in','borough','llp','royal','on','communications','social','st','hospitals',
#                     'office','for','ft','public','holdings','agency','worldwide','scotland','university','central',
#                     'essex','design','defence','government','yorkshire','virgin','ambulance','ireland']
# stop_word_dic_1["GR"]=[]
# stop_word_dic_1["IE"]=['ireland','of']
# stop_word_dic_1["IN"]=['india','hindustan','limi','pvt','ltd','and','insurance','it','of','pr','systems','development','labs',
#                     'life']
# stop_word_dic_1["IS"]=[]
# stop_word_dic_1["IT"]=['italy','italia','spa','s.p.a.','srl','banca','logistica','data','computer','italian','group','ga','di'
#                     ,'italiane','servizi','il','de','media','informatica','societa','sa']
# stop_word_dic_1["LT"]=[]
# stop_word_dic_1["LU"]=['et','de']
# stop_word_dic_1["MO"]=[]
# stop_word_dic_1["NL"]=['netherlands','nederland','solutions','europe','rotterdam','group','gemeente','b.v.','services','amsterdam',
#                  'omgevingsdienst','europe','stichting','groep','zorg','zorggroep','bv','van','en','de','ggz','nv','care','ict',
#                     'centrum','utrecht','dienst','beheer','zuid','automotive']
# stop_word_dic_1["NO"]=['norway','norge','kommune','as','ggz','nv','avd','no','for','helse','politidistrikt','vest','republika',
#                     'fylkeskommune','asa','sarpsborg']
# stop_word_dic_1["PL"]=['poland','polska','z.o.o.','s.a.','bank','media','sp','spółka']
# stop_word_dic_1["PT"]=['sa']
# stop_word_dic_1["RO"]=[]
# stop_word_dic_1["RS"]=[]
# stop_word_dic_1["RU"]=[]
# stop_word_dic_1["SE"]=['sweden','ab','sverige','stockholm','kommun','nordic','svenska','skåne']
# stop_word_dic_1["SG"]=[]
# stop_word_dic_1["SK"]=[]
# stop_word_dic_1["SM"]=[]
# stop_word_dic_1["TR"]=['pazarlama','yatirim','gida','menkul','san','a.ş','hi̇zmetleri̇','ve','opet','türk','kredi','yapi',
#                     'tic','sağlik']
# stop_word_dic_1["VA"]=[]
# stop_word_dic_1["SI"]=[]
# stop_word_dic_1["HU"]=[]
# stop_word_dic_1["HR"]=[]
# stop_word_dic_1["CY"]=[]
# stop_word_dic_1["MT"]=[]
# stop_word_dic_1["MC"]=[]
# stop_word_dic_1["CM"]=[]
# stop_word_dic_1["LI"]=[]
# stop_word_dic_1["TN"]=[]
# stop_word_dic_1["RE"]=[]
# stop_word_dic_1["GL"]=[]
# stop_word_dic_1["ZA"]=[]
# stop_word_dic_1["GE"]=[]
# stop_word_dic_1["ME"]=[]
# stop_word_dic_1["AR"]=[]
# stop_word_dic_1["EE"]=[]
# stop_word_dic_1["AD"]=[]
# stop_word_dic_1["PH"]=[]
# stop_word_dic_1["JE"]=[]
# stop_word_dic_1["BF"]=[]
# stop_word_dic_1["AE"]=[]
# stop_word_dic_1["LV"]=[]
# stop_word_dic_1["IM"]=[]

In [58]:
stop_words_ls=[]
stop_word_dic_1={}
stop_word_dic_1["General"]=[]
stop_word_dic_1["AT"]=['it','ag','kg','gmbh','ges','wien']
stop_word_dic_1["BE"]=['bvba','stad','sa','de','la','nv','media']
stop_word_dic_1["BG"]=[]
stop_word_dic_1["CH"]=['sa','ag','schweiz','suisse','gmbh']
stop_word_dic_1["CZ"]=['czech','banka','spol','česká','cz']
stop_word_dic_1["DE"]=['gmbh','ggmbh','ag','kg','co.','co','für','st','des','der','und','nord','eg','am','von','se','ohg','die','nrw','db','kgaa','mbh','gmb','sachsen','gbr']
stop_word_dic_1["DK"]=['denmark','danish','dansk','og','aps','danmark','kommune','as']
stop_word_dic_1["EG"]=[]
stop_word_dic_1["ES"]=[]
stop_word_dic_1["FI"]=['finland','turun','oy','seudun','ab','uudenmaan','oyj','suomen']
stop_word_dic_1["FO"]=[]
stop_word_dic_1["FR"]=['sa','et','gie','groupe','de','la','da','pour','it','snc','des','se','le','sarl']
stop_word_dic_1["GB"]=['it','plc','the','gb','nhs']
stop_word_dic_1["GR"]=[]
stop_word_dic_1["IE"]=[]
stop_word_dic_1["IN"]=['limi','pvt','ltd','and','it','of','pr','systems','development','labs','life']
stop_word_dic_1["IS"]=[]
stop_word_dic_1["IT"]=['srl','ga','di','il','de','sa']
stop_word_dic_1["LT"]=[]
stop_word_dic_1["LU"]=['et','de']
stop_word_dic_1["MO"]=[]
stop_word_dic_1["NL"]=['b.v.','stichting','groep','zorg','zorggroep','bv','van','en','de','ggz','nv','care','ict']
stop_word_dic_1["NO"]=['as','ggz','nv','avd','no','for','helse','asa']
stop_word_dic_1["PL"]=['z.o.o.','s.a.']
stop_word_dic_1["PT"]=['sa']
stop_word_dic_1["RO"]=[]
stop_word_dic_1["RS"]=[]
stop_word_dic_1["RU"]=[]
stop_word_dic_1["SE"]=['ab']
stop_word_dic_1["SG"]=[]
stop_word_dic_1["SK"]=[]
stop_word_dic_1["SM"]=[]
stop_word_dic_1["TR"]=['san','a.ş','ve','opet']
stop_word_dic_1["VA"]=[]
stop_word_dic_1["SI"]=[]
stop_word_dic_1["HU"]=[]
stop_word_dic_1["HR"]=[]
stop_word_dic_1["CY"]=[]
stop_word_dic_1["MT"]=[]
stop_word_dic_1["MC"]=[]
stop_word_dic_1["CM"]=[]
stop_word_dic_1["LI"]=[]
stop_word_dic_1["TN"]=[]
stop_word_dic_1["RE"]=[]
stop_word_dic_1["GL"]=[]
stop_word_dic_1["ZA"]=[]
stop_word_dic_1["GE"]=[]
stop_word_dic_1["ME"]=[]
stop_word_dic_1["AR"]=[]
stop_word_dic_1["EE"]=[]
stop_word_dic_1["AD"]=[]
stop_word_dic_1["PH"]=[]
stop_word_dic_1["JE"]=[]
stop_word_dic_1["BF"]=[]
stop_word_dic_1["AE"]=[]
stop_word_dic_1["LV"]=[]
stop_word_dic_1["IM"]=[]

## Adding new codes into the dictionaries:

In [59]:
unique_code_ls=actual_df.code.value_counts().index.tolist()
existing_code_set=set(stop_word_dic_1.keys())
new_code_ls = [x for x in unique_code_ls if x not in existing_code_set]
print(new_code_ls)
def add_new_keys_dict(stop_words_dic):
    for new_code_idx,new_code in enumerate(new_code_ls):
        print(new_code)
        stop_words_dic.update({new_code:[]})
    return stop_words_dic

['IR', 'ML', 'QA', 'IL', 'GM', 'CW', 'PO']


In [60]:
stop_word_dic_1=add_new_keys_dict(stop_word_dic_1)
stop_words_ls.append(stop_word_dic_1)

IR
ML
QA
IL
GM
CW
PO


## Stop_words for SHEET_2

In [61]:
stop_word_dic_2={}
stop_word_dic_2["General"]=[]
stop_word_dic_2["AT"]=['gmbh','kg','gesmbh','ag']
stop_word_dic_2["BE"]=['sa','nv']
stop_word_dic_2["BG"]=[]
stop_word_dic_2["CH"]=['sa','gmbh','ag']
stop_word_dic_2["CZ"]=['czech']
stop_word_dic_2["DE"]=['gmbh','springer','kg','co','ag','it','holding','eg','mbh','gm','services']
stop_word_dic_2["DK"]=['aps']
stop_word_dic_2["EG"]=[]
stop_word_dic_2["ES"]=[]
stop_word_dic_2["FI"]=['oy','ab']
stop_word_dic_2["FO"]=[]
stop_word_dic_2["FR"]=['de','la','sas','caisse','louis']
stop_word_dic_2["GB"]=['uk','ltd','limited']
stop_word_dic_2["GR"]=[]
stop_word_dic_2["HU"]=['kft']
stop_word_dic_2["IE"]=['limited','ireland','ltd']
stop_word_dic_2["IN"]=['limited','pvt','ltd','private','insurance','company','lim','services']
stop_word_dic_2["IS"]=[]
stop_word_dic_2["IT"]=['srl','sp','legale','agricola']
stop_word_dic_2["LT"]=[]
stop_word_dic_2["LU"]=['de']
stop_word_dic_2["MO"]=[]
stop_word_dic_2["NL"]=['bv','oost','services','sales','service','se']
stop_word_dic_2["NO"]=['as','norge','og','iks','sogn']
stop_word_dic_2["PL"]=['sp']
stop_word_dic_2["PT"]=['şti']
stop_word_dic_2["RO"]=[]
stop_word_dic_2["RS"]=[]
stop_word_dic_2["RU"]=[]
stop_word_dic_2["SE"]=['ab','län']
stop_word_dic_2["SG"]=[]
stop_word_dic_2["SK"]=[]
stop_word_dic_2["SM"]=[]
stop_word_dic_2["TR"]=['kredi','şti','san']
stop_word_dic_2["VA"]=[]
stop_word_dic_2["SI"]=[]

stop_word_dic_2["HR"]=[]
stop_word_dic_2["CY"]=[]
stop_word_dic_2["MT"]=[]
stop_word_dic_2["MC"]=[]
stop_word_dic_2["CM"]=[]
stop_word_dic_2["LI"]=[]
stop_word_dic_2["TN"]=[]
stop_word_dic_2["RE"]=[]
stop_word_dic_2["GL"]=[]
stop_word_dic_2["ZA"]=[]
stop_word_dic_2["GE"]=[]
stop_word_dic_2["ME"]=[]
stop_word_dic_2["AR"]=[]
stop_word_dic_2["EE"]=[]
stop_word_dic_2["AD"]=[]
stop_word_dic_2["PH"]=[]
stop_word_dic_2["JE"]=[]
stop_word_dic_2["BF"]=[]
stop_word_dic_2["AE"]=[]
stop_word_dic_2["LV"]=[]
stop_word_dic_2["IM"]=[]
stop_word_dic_2=add_new_keys_dict(stop_word_dic_2)

stop_word_dic_2=add_new_keys_dict(stop_word_dic_2)
stop_words_ls.append(stop_word_dic_2)

IR
ML
QA
IL
GM
CW
PO
IR
ML
QA
IL
GM
CW
PO


## Stop_words for SHEET_3

In [62]:
stop_word_dic_3={}
stop_word_dic_3["General"]=[]
stop_word_dic_3["AT"]=['kg','gmbh']
stop_word_dic_3["BE"]=[]
stop_word_dic_3["BG"]=[]
stop_word_dic_3["CH"]=[]
stop_word_dic_3["CZ"]=[]
stop_word_dic_3["DE"]=['gmbh','kg']
stop_word_dic_3["DK"]=[]
stop_word_dic_3["EG"]=[]
stop_word_dic_3["ES"]=[]
stop_word_dic_3["FI"]=[]
stop_word_dic_3["FO"]=[]
stop_word_dic_3["FR"]=[]
stop_word_dic_3["GB"]=['limited','ltd','trust']
stop_word_dic_3["GR"]=[]
stop_word_dic_3["IE"]=[]
stop_word_dic_3["IN"]=['pvt','ltd','private','limited']
stop_word_dic_3["IS"]=[]
stop_word_dic_3["IT"]=[]
stop_word_dic_3["LT"]=[]
stop_word_dic_3["LU"]=[]
stop_word_dic_3["MO"]=[]
stop_word_dic_3["NL"]=[]
stop_word_dic_3["NO"]=['møre','og']
stop_word_dic_3["PL"]=['sp']
stop_word_dic_3["PT"]=[]
stop_word_dic_3["RO"]=[]
stop_word_dic_3["RS"]=[]
stop_word_dic_3["RU"]=[]
stop_word_dic_3["SE"]=[]
stop_word_dic_3["SG"]=[]
stop_word_dic_3["SK"]=[]
stop_word_dic_3["SM"]=[]
stop_word_dic_3["TR"]=[]
stop_word_dic_3["VA"]=[]
stop_word_dic_3["SI"]=[]
stop_word_dic_3["HU"]=[]
stop_word_dic_3["HR"]=[]
stop_word_dic_3["CY"]=[]
stop_word_dic_3["MT"]=[]
stop_word_dic_3["MC"]=[]
stop_word_dic_3["CM"]=[]
stop_word_dic_3["LI"]=[]
stop_word_dic_3["TN"]=[]
stop_word_dic_3["RE"]=[]
stop_word_dic_3["GL"]=[]
stop_word_dic_3["ZA"]=[]
stop_word_dic_3["GE"]=[]
stop_word_dic_3["ME"]=[]
stop_word_dic_3["AR"]=[]
stop_word_dic_3["EE"]=[]
stop_word_dic_3["AD"]=[]
stop_word_dic_3["PH"]=[]
stop_word_dic_3["JE"]=[]
stop_word_dic_3["BF"]=[]
stop_word_dic_3["AE"]=[]
stop_word_dic_3["LV"]=[]
stop_word_dic_3["IM"]=[]
stop_word_dic_3=add_new_keys_dict(stop_word_dic_3)

stop_word_dic_3=add_new_keys_dict(stop_word_dic_3)
stop_words_ls.append(stop_word_dic_3)

IR
ML
QA
IL
GM
CW
PO
IR
ML
QA
IL
GM
CW
PO


## Stop_words for SHEET_4

In [63]:
stop_word_dic_4={}
stop_word_dic_4["General"]=[]
stop_word_dic_4["AT"]=[]
stop_word_dic_4["BE"]=[]
stop_word_dic_4["BG"]=[]
stop_word_dic_4["CH"]=[]
stop_word_dic_4["CZ"]=[]
stop_word_dic_4["DE"]=['gmbh']
stop_word_dic_4["DK"]=[]
stop_word_dic_4["EG"]=[]
stop_word_dic_4["ES"]=[]
stop_word_dic_4["FI"]=[]
stop_word_dic_4["FO"]=[]
stop_word_dic_4["FR"]=[]
stop_word_dic_4["GB"]=[]
stop_word_dic_4["GR"]=[]
stop_word_dic_4["IE"]=[]
stop_word_dic_4["IN"]=['pvt','ltd']
stop_word_dic_4["IS"]=[]
stop_word_dic_4["IT"]=[]
stop_word_dic_4["LT"]=[]
stop_word_dic_4["LU"]=[]
stop_word_dic_4["MO"]=[]
stop_word_dic_4["NL"]=[]
stop_word_dic_4["NO"]=[]
stop_word_dic_4["PL"]=[]
stop_word_dic_4["PT"]=[]
stop_word_dic_4["RO"]=[]
stop_word_dic_4["RS"]=[]
stop_word_dic_4["RU"]=[]
stop_word_dic_4["SE"]=[]
stop_word_dic_4["SG"]=[]
stop_word_dic_4["SK"]=[]
stop_word_dic_4["SM"]=[]
stop_word_dic_4["TR"]=[]
stop_word_dic_4["VA"]=[]
stop_word_dic_4["SI"]=[]
stop_word_dic_4["HU"]=[]
stop_word_dic_4["HR"]=[]
stop_word_dic_4["CY"]=[]
stop_word_dic_4["MT"]=[]
stop_word_dic_4["MC"]=[]
stop_word_dic_4["CM"]=[]
stop_word_dic_4["LI"]=[]
stop_word_dic_4["TN"]=[]
stop_word_dic_4["RE"]=[]
stop_word_dic_4["GL"]=[]
stop_word_dic_4["ZA"]=[]
stop_word_dic_4["GE"]=[]
stop_word_dic_4["ME"]=[]
stop_word_dic_4["AR"]=[]
stop_word_dic_4["EE"]=[]
stop_word_dic_4["AD"]=[]
stop_word_dic_4["PH"]=[]
stop_word_dic_4["JE"]=[]
stop_word_dic_4["BF"]=[]
stop_word_dic_4["AE"]=[]
stop_word_dic_4["LV"]=[]
stop_word_dic_4["IM"]=[]
stop_word_dic_4=add_new_keys_dict(stop_word_dic_4)

stop_word_dic_4=add_new_keys_dict(stop_word_dic_4)
stop_words_ls.append(stop_word_dic_4)

IR
ML
QA
IL
GM
CW
PO
IR
ML
QA
IL
GM
CW
PO


## Writing the list of dictionaries as an excel file.

In [64]:
# pd.DataFrame.from_dict(stop_word_dic)
path="/Users/simranpal20gmail.com/Documents/Apple/Data/"+Week_number+"/Manual_Match/Corpus"
os.chdir(path)
BuzzWord_write = pd.ExcelWriter('Buzz_Words.xlsx')
for index,stop_word_dic in enumerate(stop_words_ls):
    key_to_value_lengths = {k:len(v) for k, v in stop_word_dic.items()}
    maximum = max(key_to_value_lengths, key=key_to_value_lengths.get)
    # print(maximum, key_to_value_lengths[maximum])
    min_length = key_to_value_lengths[maximum]
    key_to_value_lengths = {k:len(v) for k, v in stop_word_dic.items()}
    stop_words_df = pd.DataFrame({k:pd.Series(v[:min_length]) for k,v in stop_word_dic.items()})
    stop_words_df.to_excel(BuzzWord_write, sheet_name=num2words(index+1),index=False)
BuzzWord_write.save()

<ipython-input-64-641288ab1c61>:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stop_words_df = pd.DataFrame({k:pd.Series(v[:min_length]) for k,v in stop_word_dic.items()})


In [65]:
# ini_string="3 sdc sd7f 87hbbsd 4545"
# # re.sub('[\W_]+', '', ini_string)
# # print(re.sub(r"\b\d+\b *|\b[a-z]\b *","",ini_string))

In [66]:
# code="DE"
# FINAL_STOPWORDS_LS=stop_word_dic["General"]+stop_word_dic[code]
# # df1[df1.code==code]['Names'] = df1[df1.code==code]['Names'].apply(lambda x: ' '.join([item for item in x.split() if item not in FINAL_STOPWORDS_LS]))
# df1.loc[df1['code'] == code, 'Names']=df1[df1.code==code]['Names'].apply(lambda x: ' '.join([item for item in x.split() if item not in FINAL_STOPWORDS_LS]))
# df1[df1.code==code][['Names','code']]

In [67]:
# s1="autoli cankor oto emn si"
# s2="ram si̇gorta a.ş."
# set.intersection(set(s1.split(" ")), set(s2.split(" ")))

# def get_words(input):
#     return re.compile('\w+').findall(input)

# my_string=df1[df1.code=="TR"]['Names'].tolist()
# my_string_words_set = [set(get_words(my_one_string)) for my_one_string in my_string]
# print(my_string_words_set)

In [68]:

# main_string=my_string_words_set[0]
# x="eureko si̇gorta a.ş"
# x.split(" ")
# main_string.intersection(x)
# set.intersection(set(s1.split(" ")), set(s2.split(" ")))

# x="eureko si̇gorta a.ş"
# x="DOĞAN E DÖNÜŞÜM HİZ."
# x="U F P (UK) Ltd"
# x="y-en-a.com sàrl"


# set(get_words(x))
# set(x.split(" "))
# re.sub("[^\w' ]", "", x).split()
# [''.join(c for c in word if c.isalpha() or c=="'") for word in x.split()]

# regex = r"((?:[A-Za-z](\.))*[A-Za-z]+)\.?"
# subst = "\\1\\2"
# result = list(re.sub(regex, subst, test_str, 0, re.MULTILINE).split())
# print(result)
# st1='E. M. ACCOUNTING S.R.L.'

# re.compile('\w+').findall(st1)

## Reusable Functions

In [69]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

# ------------------------------------------------------------------------------------------------------------
def set_first_index_match(main_string,x):
    try:
        if(main_string[0]==x[0]):
            return True
        else:
            return False
    except:
        return False   

# ------------------------------------------------------------------------------------------------------------
def get_words(input):
#     regex = r"((?:[A-Za-z](\.))*[A-Za-z]+)\.?"
#     subst = "\\1\\2"
#     return list(re.sub(regex, subst, test_str, 0, re.MULTILINE).split())
    return re.compile('\w+').findall(input)

# ------------------------------------------------------------------------------------------------------------
# def all_match_main_list(tars_str,pch_str):
#     tars_str_re = re.compile(tars_str)
#     if tars_str_re.match(pch_str):
#         return True
#     else:
#         return False
    
def all_match_main_list(string, pattern):
    pattern = pattern.lower().split()
    words = set(sub(r"[^a-z0-9 ]", "", string.lower()).split())  # Sanitize input
    return all(get_close_matches(word, words, cutoff=0.8) for word in pattern)

## Main Logic Code

## Without Buzzwords

In [70]:
def manual_match_without_buzz(code,number_match,df1_clean,actual_df):
    num_matches = number_match
    exclude_index_ls=[]
    FINAL_STOPWORDS_LS=stop_words_ls[num_matches-1]["General"]+stop_words_ls[num_matches-1][code]
    df1=df1_clean.copy()
    actual_df1=actual_df.copy()

#     df1.loc[df1['code'] == code, 'Names'] = df1[df1.code==code]['Names'].apply(lambda x: ' '.join([item for item in x.split() if item not in FINAL_STOPWORDS_LS]))
#     df1.loc[df1['code'] == code, 'Names'] = df1[df1.code==code]['Names'].apply(lambda x: ' '.join([item for item in x.split()]))
    my_string=df1[df1.code==code]['Names'].tolist()
    my_list=df2[df2.Country_code==code]['Child Name'].tolist()
    COLUMN_NAMES=['Country Code','Unmatched Full Cx Name','Unmatched Customer Name','PCH Customer Name',
                  'PCH Parent Name','Nice ID','STATUS','MATCH STATUS','Sum Rpt Qty','Transactions Count']
    main_df = pd.DataFrame(columns=COLUMN_NAMES)

#     my_string_words_set = [set(get_words(my_one_string)) for my_one_string in my_string]
#     my_list_words_set = [set(get_words(x)) for x in my_list]
    my_string_words_set = [get_words(my_one_string) for my_one_string in my_string]
    my_list_words_set = [get_words(x) for x in my_list]
    
    for my_string_index,main_string in enumerate(my_string_words_set):
        result = [i for i,x in enumerate(my_list_words_set) if len(set(main_string).intersection(set(x))) >= num_matches and set_first_index_match(main_string,x)]
        if(num_matches==1):
            match_with=2
        else:
            match_with=3
        if len(result)>match_with:
            changed_result_ls=[]
            tars_str=my_string[my_string_index]
            for result_index,result_index_val in enumerate(result):
                pch_matched_child_name_str=df2[(df2['Country_code'] == code)].iloc[result_index_val]['Child Name']
                pch_matched_child_name_str=pch_matched_child_name_str.replace("-", "")
                if all_match_main_list(pch_matched_child_name_str,tars_str):
                    changed_result_ls.append(result_index_val)
            result=changed_result_ls
        if result:
            accessed_mapping = map(my_list.__getitem__, result)
            accessed_list = list(accessed_mapping)
            part_df=pd.DataFrame([{'Country Code': df1[(df1['code'] == code)].iloc[my_string_index]['code'],
                                   'Unmatched Full Cx Name': actual_df1[(actual_df1['code'] == code)].iloc[my_string_index]['Names'],
                                   'Nice ID':df2[(df2['Country_code'] == code)].iloc[result]['Child CRM NICE ID'].tolist(),
                                   'Unmatched Customer Name': df1[(df1['code'] == code)].iloc[my_string_index]['Names'],
                                   'PCH Customer Name': df2[(df2['Country_code'] == code)].iloc[result]['Child Name'].tolist(),
                                   'PCH Parent Name': df2[(df2['Country_code'] == code)].iloc[result]['Parent Name'].tolist(),
                                   'STATUS': df1[(df1['code'] == code)].iloc[my_string_index]['STATUS'],
                                   'MATCH STATUS': df1[(df1['code'] == code)].iloc[my_string_index]['MATCH STATUS'],
                                   'Sum Rpt Qty': df1[(df1['code'] == code)].iloc[my_string_index]['Sum_Rpt_Qty'],
                                   'Transactions Count': df1[(df1['code'] == code)].iloc[my_string_index]['Transaction_Counts']
                                  }])
            main_df=main_df.append(part_df)
            
            main_index=df1_clean[(df1_clean['code'] == code)].iloc[[my_string_index]].index
            
            exclude_index_ls.append(main_index.tolist()[0])
    df1_clean=df1_clean[~df1_clean.index.isin(exclude_index_ls)].copy()
    actual_df=actual_df[~actual_df.index.isin(exclude_index_ls)].copy()

    if main_df.empty:
        return (main_df,df1_clean,actual_df)
    else:
        exploded_main_df=explode(main_df, ['PCH Customer Name','PCH Parent Name','Nice ID'], fill_value='')
        return (exploded_main_df,df1_clean,actual_df)

## With Buzzwords

In [71]:
def manual_match(code,number_match,df1_clean,actual_df):
    num_matches = number_match
    exclude_index_ls=[]
    FINAL_STOPWORDS_LS=stop_words_ls[num_matches-1]["General"]+stop_words_ls[num_matches-1][code]
    df1=df1_clean.copy()
    actual_df1=actual_df.copy()

    df1.loc[df1['code'] == code, 'Names'] = df1[df1.code==code]['Names'].apply(lambda x: ' '.join([item for item in x.split() if item not in FINAL_STOPWORDS_LS]))
    my_string=df1[df1.code==code]['Names'].tolist()
    my_list=df2[df2.Country_code==code]['Child Name'].tolist()
    COLUMN_NAMES=['Country Code','Unmatched Full Cx Name','Nice ID','Unmatched Customer Name','PCH Customer Name',
                  'PCH Parent Name','STATUS','MATCH STATUS','Sum Rpt Qty','Transactions Count']
    main_df = pd.DataFrame(columns=COLUMN_NAMES)

#     my_string_words_set = [set(get_words(my_one_string)) for my_one_string in my_string]
#     my_list_words_set = [set(get_words(x)) for x in my_list]
    my_string_words_set = [get_words(my_one_string) for my_one_string in my_string]
    my_list_words_set = [get_words(x) for x in my_list]
    
    for my_string_index,main_string in enumerate(my_string_words_set):
        result = [i for i,x in enumerate(my_list_words_set) if len(set(main_string).intersection(set(x))) >= num_matches and set_first_index_match(main_string,x)]
        if len(result)>3:
            changed_result_ls=[]
            tars_str=my_string[my_string_index]
            for result_index,result_index_val in enumerate(result):
                pch_matched_child_name_str=df2[(df2['Country_code'] == code)].iloc[result_index_val]['Child Name']
#                 pch_matched_child_name_str=pch_matched_child_name_str.replace("-", "")
                if all_match_main_list(pch_matched_child_name_str,tars_str):
                    changed_result_ls.append(result_index_val)
            result=changed_result_ls
        if result:
            accessed_mapping = map(my_list.__getitem__, result)
            accessed_list = list(accessed_mapping)
            part_df=pd.DataFrame([{'Country Code': df1[(df1['code'] == code)].iloc[my_string_index]['code'],
                                   'Unmatched Full Cx Name': actual_df1[(actual_df1['code'] == code)].iloc[my_string_index]['Names'],
                                   'Nice ID':df2[(df2['Country_code'] == code)].iloc[result]['Child CRM NICE ID'].tolist(),
                                   'Unmatched Customer Name': df1[(df1['code'] == code)].iloc[my_string_index]['Names'],
                                   'PCH Customer Name': df2[(df2['Country_code'] == code)].iloc[result]['Child Name'].tolist(),
                                   'PCH Parent Name': df2[(df2['Country_code'] == code)].iloc[result]['Parent Name'].tolist(),
                                   'STATUS': df1[(df1['code'] == code)].iloc[my_string_index]['STATUS'],
                                   'MATCH STATUS': df1[(df1['code'] == code)].iloc[my_string_index]['MATCH STATUS'],
                                   'Sum Rpt Qty': df1[(df1['code'] == code)].iloc[my_string_index]['Sum_Rpt_Qty'],
                                   'Transactions Count': df1[(df1['code'] == code)].iloc[my_string_index]['Transaction_Counts']
                                  }])
            main_df=main_df.append(part_df)
            
            main_index=df1_clean[(df1_clean['code'] == code)].iloc[[my_string_index]].index
            
            exclude_index_ls.append(main_index.tolist()[0])
    df1_clean=df1_clean[~df1_clean.index.isin(exclude_index_ls)].copy()
    actual_df=actual_df[~actual_df.index.isin(exclude_index_ls)].copy()
    main_df.insert(loc=2, column='Flag', value=0)
#     main_df['Flag']=0
    if main_df.empty:
        return (main_df,df1_clean,actual_df)
    else:
        exploded_main_df=explode(main_df, ['PCH Customer Name','PCH Parent Name','Nice ID'], fill_value='')
        return (exploded_main_df,df1_clean,actual_df)

In [72]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', 3)
# pd.set_option('display.width', 0)
# pd.set_option('display.max_colwidth', 29)
# exploded_df,df1_clean,actual_df=manual_match_without_buzz('DE',2,df1_clean,actual_df)
# # exploded_df,df1_clean,actual_df=manual_match('IN',4,df1_clean,actual_df)
# # print(manual_match('PT',4,df1_clean,actual_df))

In [73]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', 3)
# pd.set_option('display.width', 0)
# pd.set_option('display.max_colwidth', 29)
# print(df1_clean.shape)
# print(actual_df.shape)
# exploded_df,df1_clean,actual_df=manual_match('IN',3,df1_clean,actual_df)
# print(df1_clean.shape)
# print(actual_df.shape)

In [74]:
# test_df=pd.DataFrame({'code':['AT','IN','IN','IN','DE'],'col2':[0,1,2,3,4]})
# print(test_df)
# exclude_index_ls=[1,4]
# code='IN'

## Writing the final "Appropriate Matched File" in an excel format 

In [75]:
# path='/Users/simranpal20gmail.com/Documents/Apple/Data/Week14/Manual_Match'
path="/Users/simranpal20gmail.com/Documents/Apple/Data/"+Week_number+"/Manual_Match"
os.chdir(path)
unique_country_code_ls=actual_df.code.value_counts().index.tolist()
unique_country_code_ls
COLUMN_NAMES=['Country Code','Unmatched Full Cx Name','Flag','Nice ID','Unmatched Customer Name','PCH Customer Name',
                  'PCH Parent Name','STATUS','MATCH STATUS','Sum Rpt Qty','Transactions Count']
# df1_clean=df1.copy()
Ciara = pd.ExcelWriter('Manual_Match_Ciara.xlsx')
Neil = pd.ExcelWriter('Manual_Match_Neil.xlsx')
Simran = pd.ExcelWriter('Manual_Match_Simran.xlsx')

for matched_no in range(4, 0, -1):
    country_main_df = pd.DataFrame(columns=COLUMN_NAMES)
    for cc in unique_country_code_ls:
        exploded_df,df1_clean,actual_df=manual_match(cc,matched_no,df1_clean,actual_df)
        country_main_df=country_main_df.append(exploded_df)
    admins_divide_np=np.array_split(country_main_df, admins_divide)
    for divide in range(0,admins_divide):
        dynamic_var=admins_divide_dict[divide+1]
        admins_divide_np[divide].to_excel(eval(dynamic_var), sheet_name=num2words(matched_no),index=False)
Ciara.save()
Neil.save()
Simran.save()

In [76]:
# x1=['media','impact','co','kg']
# sorted(set(x1), key=x.index)

# # x1={'media','impact','co','kg'}
# # x2={'media','impact','gmbh','co.','kg'}
# # for e in x2:
# #     print(e)
# # print(list(x2))
# # set_first_index_match(x1,x2)

# x1='alation india private limited'
# x2='media impact gmbh co. kg'
# get_words(x2)
# # sorted(set(get_words(x2)), key=get_words(x2).index)

## Writing the file

# ---------------------------------END HERE-----------------------------------